In [1]:
## import libraries
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from collections import Counter
import re
from scipy.stats import zscore, shapiro
from sklearn.feature_selection import mutual_info_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\xiaow\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Step 1. Exploratory Data Analysis (EDA)

### Step 1.1. Load the training dataset and perform an initial data inspection

In [2]:
print("Loading training data...")
train_df = pd.read_csv('C:/Users/xiaow/work/jp_awm/interview/Take Home Project/training_loan_data.csv', header=1)
ind_col = 'train_indicator'
train_df[ind_col] = 1.0
print(f"Training data shape: {train_df.shape}")
y_col = 'bad_flag'
print("dependent variable is: " + y_col)
train_df.head()

Loading training data...
Training data shape: (199121, 24)
dependent variable is: bad_flag


,id,member_id,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,desc,purpose,...,mths_since_recent_inq,revol_util,total_bc_limit,mths_since_last_major_derog,tot_hi_cred_lim,tot_cur_bal,application_approved_flag,internal_score,bad_flag,train_indicator
0,10000001,11983056.0,7550,36 months,16.24%,3 years,RENT,28000.0,NaN,debt_consolidation,...,17.0,72%,4000.0,NaN,3828.953801,5759.0,1,99,0.0,1.0
1,10000002,12002921.0,27050,36 months,10.99%,10+ years,OWN,55000.0,Borrower added on 12/31/13 > Combining high ...,debt_consolidation,...,8.0,61.20%,35700.0,NaN,34359.940730,114834.0,1,353,0.0,1.0
2,10000003,11983096.0,12000,36 months,10.99%,4 years,RENT,60000.0,Borrower added on 12/31/13 > I would like to...,debt_consolidation,...,3.0,24%,18100.0,NaN,16416.617760,7137.0,1,157,0.0,1.0
3,10000004,12003142.0,28000,36 months,7.62%,5 years,MORTGAGE,325000.0,NaN,debt_consolidation,...,3.0,54.60%,42200.0,NaN,38014.149760,799592.0,1,365,0.0,1.0
4,10000005,11993233.0,12000,36 months,13.53%,10+ years,RENT,40000.0,NaN,debt_consolidation,...,17.0,68.80%,7000.0,53.0,6471.462236,13605.0,1,157,0.0,1.0


In [3]:
print("Loading testing data...")
test_df = pd.read_csv('C:/Users/xiaow/work/jp_awm/interview/Take Home Project/testing_loan_data.csv', header=0)
test_df[ind_col] = 0.0
print(f"Testing data shape: {test_df.shape}")
test_df.head()

Loading testing data...
Testing data shape: (102505, 24)


,id,member_id,loan_amnt,term,int_rate,emp_length,home_ownership,annual_inc,desc,purpose,...,mths_since_recent_inq,revol_util,total_bc_limit,mths_since_last_major_derog,tot_hi_cred_lim,tot_cur_bal,application_approved_flag,internal_score,bad_flag,train_indicator
0,20000001,22419852,10000,36 months,22.15%,8 years,RENT,37000.0,NaN,debt_consolidation,...,3.0,73.10%,16200,NaN,14877.170280,36809,1,131,NaN,0.0
1,20000002,22349118,1400,36 months,18.24%,6 years,RENT,41000.0,NaN,other,...,9.0,11.50%,4000,NaN,4097.304770,19536,1,19,NaN,0.0
2,20000003,22398818,7000,36 months,12.49%,3 years,RENT,68900.0,NaN,debt_consolidation,...,11.0,48.10%,11900,80.0,12688.495160,241465,1,92,NaN,0.0
3,20000004,22419015,18000,60 months,16.29%,9 years,MORTGAGE,41000.0,NaN,debt_consolidation,...,0.0,38.10%,7600,73.0,7908.799817,179757,1,235,NaN,0.0
4,20000005,22388614,12000,36 months,12.99%,10+ years,MORTGAGE,64000.0,NaN,home_improvement,...,NaN,57.90%,21000,NaN,19378.561060,31953,1,157,NaN,0.0


In [4]:
print("\n=== Train Data Types and Missing Values ===")
train_df.info()


=== Train Data Types and Missing Values ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 199121 entries, 0 to 199120
Data columns (total 24 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           199121 non-null  int64  
 1   member_id                    189457 non-null  float64
 2   loan_amnt                    199121 non-null  int64  
 3   term                         189457 non-null  object 
 4   int_rate                     189457 non-null  object 
 5   emp_length                   181531 non-null  object 
 6   home_ownership               189457 non-null  object 
 7   annual_inc                   189457 non-null  float64
 8   desc                         82004 non-null   object 
 9   purpose                      189457 non-null  object 
 10  percent_bc_gt_75             180419 non-null  float64
 11  bc_util                      180333 non-null  float64
 12  dti          

In [5]:
print("\n=== Test Data Types and Missing Values ===")
test_df.info()


=== Test Data Types and Missing Values ===
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102505 entries, 0 to 102504
Data columns (total 24 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   id                           102505 non-null  int64  
 1   member_id                    102505 non-null  int64  
 2   loan_amnt                    102505 non-null  int64  
 3   term                         102505 non-null  object 
 4   int_rate                     102505 non-null  object 
 5   emp_length                   97184 non-null   object 
 6   home_ownership               102505 non-null  object 
 7   annual_inc                   102505 non-null  float64
 8   desc                         15194 non-null   object 
 9   purpose                      102505 non-null  object 
 10  percent_bc_gt_75             101459 non-null  float64
 11  bc_util                      101463 non-null  float64
 12  dti           

### Step 1.2. Data Cleaning and Visualization

#### Step 1.2.1. Remove rows in training data with missing dependent (i.e., 5% rows with missing y)

In [6]:
## the logic is that the rows in the training dataset with missing dependent variable will
## not contribute to the machine learning model training process and thus have to be removed

In [7]:
train_df = train_df.loc[train_df['bad_flag'].notnull(), :].copy()
print('shape of dataframe with non-missing dependent variable')
print(train_df.shape)

shape of dataframe with non-missing dependent variable
(189457, 24)


#### Step 1.2.2. Drop duplicates

In [8]:
## logic is that add the duplicated rows into the model will not boost the training or prediction
## performance. It will overweight certain observations

In [9]:
print('shape before drop duplicates')
print(train_df.shape)
train_df = train_df.drop_duplicates()
print('shape after drop duplicates')
print(train_df.shape)

shape before drop duplicates
(189457, 24)
shape after drop duplicates
(188123, 24)


#### Step 1.2.3. Combine train and test

In [10]:
combined_df = pd.concat([train_df, test_df], axis=0)
print(len(combined_df) - len(train_df) - len(test_df))
combined_df.shape

0


(290628, 24)

#### Step 1.2.3. Drop IDs

In [11]:
## ID columns don’t carry useful patterns for prediction and can hurt model performance,
## e.g., overfitting or inflated feature importance

In [12]:
print(combined_df.shape)
train_df = combined_df.loc[combined_df[ind_col] == 1, :].copy()
print(train_df.shape)
for x in ['member_id', 'id']:
    print(x)
    assert train_df[x].nunique() == len(train_df)
    assert test_df[x].nunique() == len(test_df)
    del combined_df[x]
print(combined_df.shape)
print(train_df.shape)

(290628, 24)
(188123, 24)
member_id
id
(290628, 22)
(188123, 24)


#### Step 1.2.4. Drop columns that contains single values

In [13]:
## this kind of columns does not provide any useful information, and thus need to be removed, i.e., application_approved_flag

In [14]:
train_df = combined_df.loc[combined_df[ind_col] == 1, :].copy()
print(train_df.shape)
problem_cols = set([x for x in train_df.columns if train_df[x].nunique() == 1]).difference(set([ind_col]))
print(problem_cols)
print(combined_df.shape)
for x in problem_cols:
    del combined_df[x]
print(combined_df.shape)

(188123, 22)
{'application_approved_flag'}
(290628, 22)
(290628, 21)


#### Step 1.2.5. Drop columns with missing rate too high

In [15]:
## idea is that if the columns has too many missings, and it does not have
## high correlation with dependent variable, then it does not contribute to
## the model training process

In [16]:
train_df = combined_df.loc[combined_df[ind_col] == 1, :].copy()
print(train_df.shape)
for x in train_df.columns:
    missing_rate = 100.0 * train_df[x].isnull().sum() / len(train_df)
    print(x + ' ' + str(missing_rate))

(188123, 21)
loan_amnt 0.0
term 0.0
int_rate 0.0
emp_length 4.1871541491471005
home_ownership 0.0
annual_inc 0.0
desc 56.69801140743024
purpose 0.0
percent_bc_gt_75 4.798456329103831
bc_util 4.843107966596323
dti 0.0
inq_last_6mths 0.0
mths_since_recent_inq 14.809459768343052
revol_util 0.06644588912573157
total_bc_limit 3.984095511978865
mths_since_last_major_derog 82.72566352864881
tot_hi_cred_lim 3.984095511978865
tot_cur_bal 14.746203281895356
internal_score 0.0
bad_flag 0.0
train_indicator 0.0


In [17]:
print(combined_df.shape)
del combined_df['mths_since_last_major_derog']
print(combined_df.shape)

(290628, 21)
(290628, 20)


#### Step 1.2.6. Convert int columns to float

In [18]:
## idea is that those int columns, loan_amnt and internal_score, are numerically comparable, 
## e.g., loan_amnt = 10 vs. loan_amnt = 20 means the 2nd one has twice the amount, thus make sense to convert to float

In [19]:
train_df = combined_df.loc[combined_df[ind_col] == 1, :].copy()
print(train_df.shape)
print(combined_df.shape)
int_cols = train_df.select_dtypes(include='int64').columns
for x in int_cols:
    print(x)
    combined_df[x] = combined_df[x].astype(float)
print(combined_df.shape)

(188123, 20)
(290628, 20)
loan_amnt
internal_score
(290628, 20)


#### Step 1.2.7 Convert some objective columns to float

In [20]:
## idea is that there are some columns ending with %, which should be numerical values, convert them from string to numerical

In [21]:
def convert_specified_percent_columns(df, percent_cols, divide_by_100=True):
    """
    Convert specified columns with values ending in '%' to floats.
    
    Parameters:
    -----------
    df : pandas DataFrame
        Input dataset.
    percent_cols : list
        List of column names to convert.
    divide_by_100 : bool, default=True
        If True, converts percentages to fractions (e.g., '50%' -> 0.5).
        If False, keeps numeric values as is (e.g., '50%' -> 50.0).
    
    Returns:
    --------
    df_copy : pandas DataFrame
        DataFrame with specified percentage columns converted to float.
    """
    df_copy = df.copy()
    
    for col in percent_cols:
        if col in df_copy.columns:
            df_copy[col] = df_copy[col].astype(str).str.rstrip('%').astype(float)
            if divide_by_100:
                df_copy[col] = df_copy[col] / 100.0
            print(f"Converted column '{col}' from percentage string to float.")
        else:
            print(f"Warning: Column '{col}' not found in DataFrame.")
    
    return df_copy

train_df = combined_df.loc[combined_df[ind_col] == 1, :].copy()
print(train_df.shape)
print(combined_df.shape)
combined_df = convert_specified_percent_columns(combined_df, percent_cols=['int_rate', 'revol_util'], divide_by_100=True)
train_df = combined_df.loc[combined_df[ind_col] == 1, :].copy()
print(train_df.shape)
print(combined_df.shape)

(188123, 20)
(290628, 20)
Converted column 'int_rate' from percentage string to float.
Converted column 'revol_util' from percentage string to float.
(188123, 20)
(290628, 20)


#### Step 1.2.8. Deal with missing values

In [22]:
## idea is that desc has lots of missing values but it contains useful information and thus should not be removed, first ignore 
## desc, and for the rest of the dependent variables, use iterative imputer to impute missing values in numerical columns.
## for category columns, simply using mode to impute (do not use dependent!!)

In [23]:
def missing_percentage(df):
    """
    Print percentage of missing values for all variables in a DataFrame.
    """
    missing = df.isnull().sum()
    perc_missing = (missing / len(df)) * 100
    result = pd.DataFrame({
        'Missing_Count': missing,
        'Missing_Percentage': perc_missing
    })
    result = result[result['Missing_Count'] > 0].sort_values(by='Missing_Percentage', ascending=False)
    return result

train_df = combined_df.loc[combined_df[ind_col] == 1, :].copy()
print(train_df.shape)
aa = missing_percentage(train_df)
print(train_df.shape)
aa

(188123, 20)
(188123, 20)


,Missing_Count,Missing_Percentage
desc,106662,56.698011
mths_since_recent_inq,27860,14.809460
tot_cur_bal,27741,14.746203
bc_util,9111,4.843108
percent_bc_gt_75,9027,4.798456
emp_length,7877,4.187154
total_bc_limit,7495,3.984096
tot_hi_cred_lim,7495,3.984096
revol_util,125,0.066446


In [24]:
def impute_mixed_data_with_train_indicator(df, train_indicator='train_indicator', exclude_cols=['desc', 'bad_flag'], add_missing_ind=True):
    """
    Impute missing values for a combined train+test dataset using train_indicator.
    Fits imputers on train data and applies to full dataset.
    Handles numeric (Iterative Imputer) and categorical (mode) columns separately.
    """
    df_copy = df.copy()
    
    # Split train/test
    train_data = df_copy[df_copy[train_indicator] == 1]
    
    # Numeric columns (excluding target and text columns)
    numeric_cols = df_copy.select_dtypes(include=[np.number]).columns.tolist()
    num_impute_cols = [c for c in numeric_cols if c not in exclude_cols + [train_indicator]]
    
    # Categorical columns (object, category)
    cat_cols = df_copy.select_dtypes(exclude=[np.number]).columns.tolist()
    cat_impute_cols = [c for c in cat_cols if c not in exclude_cols + [train_indicator]]
    
    # Add missing indicators for numeric columns
    if add_missing_ind:
        for col in num_impute_cols:
            if df_copy[col].isnull().any():
                df_copy[f'{col}_missing_ind'] = df_copy[col].isnull().astype(int)
    
    # ✅ Numeric: Fit Iterative Imputer on train, transform all
    if len(num_impute_cols) > 0:
        num_imputer = IterativeImputer(random_state=42)
        num_imputer.fit(train_data[num_impute_cols])
        df_copy[num_impute_cols] = num_imputer.transform(df_copy[num_impute_cols])
    
    # ✅ Categorical: Fit SimpleImputer (mode) on train, transform all
    if len(cat_impute_cols) > 0:
        cat_imputer = SimpleImputer(strategy='most_frequent')
        cat_imputer.fit(train_data[cat_impute_cols])
        df_copy[cat_impute_cols] = cat_imputer.transform(df_copy[cat_impute_cols])
    
    return df_copy, num_imputer, cat_imputer

train_df = combined_df.loc[combined_df[ind_col] == 1, :].copy()
print(train_df.shape)
print(combined_df.shape)
combined_df, _, _ = impute_mixed_data_with_train_indicator(df=combined_df, train_indicator='train_indicator', exclude_cols=['desc', 'bad_flag', 'train_indicator'], add_missing_ind=True)
train_df = combined_df.loc[combined_df[ind_col] == 1, :].copy()
print(train_df.shape)
print(combined_df.shape)

(188123, 20)
(290628, 20)
(188123, 27)
(290628, 27)


In [25]:
missing_percentage(combined_df)

,Missing_Count,Missing_Percentage
desc,193973,66.742709
bad_flag,102505,35.270174


#### Step 1.2.9. For float independent variables, add log, ^1/2, ^1/3, ^1/4, ^1/5

In [26]:
### idea is to make the numbers smaller to make the data more stable and not affected too much by outliers

In [27]:
def safe_cuberoot(series):
    """Compute cube root for a pandas Series, handling negatives correctly."""
    return series.apply(lambda x: -1 * np.power(-x, 1.0/3.0) if x < 0 else np.power(x, 1.0/3.0))

def safe_5rt(series):
    """Compute cube root for a pandas Series, handling negatives correctly."""
    return series.apply(lambda x: -1 * np.power(-x, 1.0/5.0) if x < 0 else np.power(x, 1.0/5.0))

def create_float_transformations(df, excl_vars=None):
    """
    Create transformation columns for float columns based on value ranges:
    - If all > 0: log(x), sqrt(x), x^(1/3), x^(1/4), x^(1/5)
    - If all >= 0: log(x+1), sqrt(x), x^(1/3), x^(1/4), x^(1/5)
    - If any < 0: only x^(1/3), x^(1/5) and print % of negatives

    Parameters:
    -----------
    df : pandas DataFrame
        Input dataframe.
    excl_vars : list or None
        Columns to exclude (e.g., target variable).

    Returns:
    --------
    df_copy : pandas DataFrame
        Dataframe with added transformation columns.
    """
    print("Shape before:", df.shape)
    df_copy = df.copy()

    # Identify numeric columns
    float_cols = df_copy.select_dtypes(include=[np.number]).columns.tolist()
    if excl_vars:
        float_cols = [c for c in float_cols if c not in excl_vars]

    # ✅ Ensure no missing values in selected numeric columns
    assert df_copy[float_cols].isnull().sum().sum() == 0, "NaN values detected in numeric columns. Please impute first."

    for col in float_cols:
        col_data = df_copy[col]
        min_val = col_data.min()

        if min_val > 0:
            # All values > 0: safe for log(x)
            df_copy[f'{col}_log'] = np.log(df_copy[col])
            df_copy[f'{col}_sqrt'] = np.power(df_copy[col], 1.0/2.0)
            df_copy[f'{col}_cbrt'] = np.power(df_copy[col], 1.0/3.0)
            df_copy[f'{col}_4rt'] = np.power(df_copy[col], 1.0/4.0)
            df_copy[f'{col}_5rt'] = np.power(df_copy[col], 1.0/5.0)

        elif min_val >= 0:
            # All values >= 0: use log(x+1)
            df_copy[f'{col}_log'] = np.log(df_copy[col] + 1)
            df_copy[f'{col}_sqrt'] = np.power(df_copy[col], 1.0/2.0)
            df_copy[f'{col}_cbrt'] = np.power(df_copy[col], 1.0/3.0)
            df_copy[f'{col}_4rt'] = np.power(df_copy[col], 1.0/4.0)
            df_copy[f'{col}_5rt'] = np.power(df_copy[col], 1.0/5.0)

        else:
            # Some values < 0: only 1/3 and 1/5 power transforms
            neg_count = (df_copy[col] < 0).sum()
            total_values = col_data.shape[0]
            perc_negative = (neg_count / total_values) * 100
            print(f"Column '{col}': {perc_negative:.2f}% of values are negative. Using only 1/3 and 1/5 transforms.")
            df_copy[f'{col}_cbrt'] = safe_cuberoot(df_copy[col])
            df_copy[f'{col}_5rt'] = safe_5rt(df_copy[col])

    print("Shape after:", df_copy.shape)
    return df_copy

train_df = combined_df.loc[combined_df[ind_col] == 1, :].copy()
print(train_df.shape)
print(combined_df.shape)
combined_df = create_float_transformations(df=combined_df, excl_vars=[y_col, ind_col, 'desc'])
train_df = combined_df.loc[combined_df[ind_col] == 1, :].copy()
print(train_df.shape)
print(combined_df.shape)

(188123, 27)
(290628, 27)
Shape before: (290628, 27)
Column 'percent_bc_gt_75': 0.08% of values are negative. Using only 1/3 and 1/5 transforms.
Column 'mths_since_recent_inq': 0.18% of values are negative. Using only 1/3 and 1/5 transforms.
Column 'tot_cur_bal': 0.03% of values are negative. Using only 1/3 and 1/5 transforms.
Shape after: (290628, 118)
(188123, 118)
(290628, 118)


In [28]:
missing_percentage(combined_df)

,Missing_Count,Missing_Percentage
desc,193973,66.742709
bad_flag,102505,35.270174


#### Step 1.2.10. handling outliers for numerical

In [29]:
## idea is that for each numerical independent variable, if it is approximately normal, then truncate using 3 stddev zscore,
## o.w., truncate use 1.5 IQR

In [30]:
def detect_and_truncate_outliers_train_test(df, train_indicator='train_indicator', y_var='bad_flag', z_thresh=3.0):
    """
    Detect outliers based on training data and apply to full dataset.
    Uses Shapiro test: if normal -> z-score; else -> IQR.
    Assumes all NaNs have been removed beforehand.
    """
    df_copy = df.copy()
    train_data = df_copy[df_copy[train_indicator] == 1]
    
    # ✅ Ensure no missing values before proceeding
    numeric_cols = [c for c in df_copy.select_dtypes(include=[np.number]).columns if c not in [y_var, train_indicator, 'desc']]
    assert df_copy[numeric_cols].isnull().sum().sum() == 0, "NaN values detected in numeric columns. Please impute first."
    
    summary = []
    
    for col in numeric_cols:
        data = train_data[col]
        
        # Shapiro-Wilk test for normality (skip if too few samples)
        stat, p_value = shapiro(data) if len(data) > 3 else (None, 0)
        
        if p_value and p_value > 0.05:
            # Normal: use z-score
            method = "z-score"
            mean_val = data.mean()
            std_val = data.std()
            lower_bound = mean_val - z_thresh * std_val
            upper_bound = mean_val + z_thresh * std_val
        else:
            # Not normal: use IQR
            method = "IQR"
            Q1 = data.quantile(0.25)
            Q3 = data.quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - 1.5 * IQR
            upper_bound = Q3 + 1.5 * IQR
        
        # Apply to all data
        outlier_mask = (df_copy[col] < lower_bound) | (df_copy[col] > upper_bound)
        n_outliers = outlier_mask.sum()
        
        if n_outliers > 0:
            cleaned_col = f"{col}_cleaned"
            indicator_col = f"{col}_outlier"
            df_copy[cleaned_col] = df_copy[col].clip(lower=lower_bound, upper=upper_bound)
            df_copy[indicator_col] = outlier_mask.astype(int)
        
        summary.append({
            "Column": col,
            "Method": method,
            "Lower_Bound": lower_bound,
            "Upper_Bound": upper_bound,
            "Num_Outliers_Total": int(n_outliers),
            "Num_Outliers_Train": int(((train_data[col] < lower_bound) | (train_data[col] > upper_bound)).sum())
        })
    
    summary_df = pd.DataFrame(summary)
    return df_copy, summary_df

train_df = combined_df.loc[combined_df[ind_col] == 1, :].copy()
print(train_df.shape)
print(combined_df.shape)
combined_df, summary_df = detect_and_truncate_outliers_train_test(df=combined_df, train_indicator='train_indicator', y_var='bad_flag', z_thresh=3.0)
train_df = combined_df.loc[combined_df[ind_col] == 1, :].copy()
print(train_df.shape)
print(combined_df.shape)

(188123, 118)
(290628, 118)
(188123, 308)
(290628, 308)


In [31]:
summary_df.head()

,Column,Method,Lower_Bound,Upper_Bound,Num_Outliers_Total,Num_Outliers_Train
0,loan_amnt,IQR,-10000.00000,38000.00000,0,0
1,int_rate,IQR,0.01945,0.26465,0,0
2,annual_inc,IQR,-18000.00000,150000.00000,12520,7501
3,percent_bc_gt_75,IQR,-57.50000,162.50000,1,0
4,bc_util,IQR,-9.30000,147.50000,14,6


#### Step 1.2.10. handling outliers for category

In [32]:
## note that home_ownership has only a few observations equal to OTHER or NONE, which are noise, impute them with most frequently 
## observed value, i.e., MORTGAGE

In [33]:
print(combined_df.loc[combined_df['train_indicator'] == 1, 'home_ownership'].value_counts())
print(combined_df['home_ownership'].value_counts())
combined_df.loc[combined_df['home_ownership'].isin(['OTHER', 'NONE']), 'home_ownership'] = 'MORTGAGE'
print(combined_df.loc[combined_df['train_indicator'] == 1, 'home_ownership'].value_counts())
print(combined_df['home_ownership'].value_counts())

home_ownership
MORTGAGE    96979
RENT        75609
OWN         15447
OTHER          46
NONE           42
Name: count, dtype: int64
home_ownership
MORTGAGE    150243
RENT        115139
OWN          25158
OTHER           46
NONE            42
Name: count, dtype: int64
home_ownership
MORTGAGE    97067
RENT        75609
OWN         15447
Name: count, dtype: int64
home_ownership
MORTGAGE    150331
RENT        115139
OWN          25158
Name: count, dtype: int64


#### Step 1.2.11. Extract information from column desc

In [34]:
## idea is that for a row that contains valid desc, get the length of the desc, then create 1000 dummy columns to count 
## how many times those keywords shows up in current desc

In [35]:
def create_text_features(df, text_col='desc', top_n=1000, train_indicator='train_indicator'):
    """
    Create text features:
    - desc_length: number of words per row.
    - Top N most common words based on TRAINING data only.
    - Word count dummy variables applied to full dataset.
    """
    df_copy = df.copy()

    # ✅ Safe length calculation with null handling
    df_copy['desc_length'] = df_copy[text_col].astype(str).apply(
        lambda x: len(re.findall(r'\b\w+\b', x)) if x.lower() != 'nan' else 0
    ).astype(int)

    # ✅ Build vocabulary using TRAINING data only (non-null rows)
    train_text = " ".join(
        df_copy.loc[df_copy[train_indicator] == 1, text_col].dropna().astype(str)
    )
    words = re.findall(r'\b\w+\b', train_text.lower())
    stop_words = set(stopwords.words('english'))
    filtered_words = [w for w in words if w not in stop_words]
    word_counts = Counter(filtered_words)
    most_common_words = [w for w, _ in word_counts.most_common(top_n)]

    # ✅ Apply word counts to full dataset (train + test)
    for word in most_common_words:
        pattern = re.compile(rf'\b{re.escape(word)}\b', re.IGNORECASE)
        df_copy[f'word_{word}'] = df_copy[text_col].astype(str).apply(
            lambda x: len(pattern.findall(x)) if x.lower() != 'nan' else 0
        )

    return df_copy, most_common_words

train_df = combined_df.loc[combined_df[ind_col] == 1, :].copy()
print(train_df.shape)
print(combined_df.shape)
combined_df, _ = create_text_features(df=combined_df, text_col='desc', top_n=1000, train_indicator='train_indicator')
train_df = combined_df.loc[combined_df[ind_col] == 1, :].copy()
print(train_df.shape)
print(combined_df.shape)

(188123, 308)
(290628, 308)
(188123, 1309)
(290628, 1309)


In [36]:
### remove desc
del combined_df['desc']

In [37]:
missing_percentage(combined_df)

,Missing_Count,Missing_Percentage
bad_flag,102505,35.270174


#### Step 1.2.12. One hot encoding all category variables and then remove them

In [38]:
def one_hot_encode_objects(df, drop_first=False):
    """
    One-hot encode all object columns in a DataFrame and drop the original columns.

    Parameters:
    -----------
    df : pandas DataFrame
        Input dataset.
    drop_first : bool, default=False
        Whether to drop the first level to avoid dummy variable trap.

    Returns:
    --------
    df_encoded : pandas DataFrame
        DataFrame with object columns one-hot encoded and original object columns removed.
    object_cols : list
        List of columns that were encoded.
    """
    df_copy = df.copy()
    
    # Identify object columns
    object_cols = df_copy.select_dtypes(include=['object']).columns.tolist()
    print(object_cols)
    
    # One-hot encode object columns
    if object_cols:
        dummies = pd.get_dummies(df_copy[object_cols], prefix=object_cols, drop_first=drop_first)
        df_copy = pd.concat([df_copy.drop(columns=object_cols), dummies], axis=1)
    
    return df_copy, object_cols

train_df = combined_df.loc[combined_df[ind_col] == 1, :].copy()
print(train_df.shape)
print(combined_df.shape)
combined_df, _ = one_hot_encode_objects(combined_df, drop_first=False)
train_df = combined_df.loc[combined_df[ind_col] == 1, :].copy()
print(train_df.shape)
print(combined_df.shape)

(188123, 1308)
(290628, 1308)
['term', 'emp_length', 'home_ownership', 'purpose']
(188123, 1333)
(290628, 1333)


In [39]:
missing_percentage(combined_df)

,Missing_Count,Missing_Percentage
bad_flag,102505,35.270174


In [40]:
combined_df.head()

,loan_amnt,int_rate,annual_inc,percent_bc_gt_75,bc_util,dti,inq_last_6mths,mths_since_recent_inq,revol_util,total_bc_limit,...,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_renewable_energy,purpose_small_business,purpose_vacation,purpose_wedding
0,7550.0,0.1624,28000.0,100.0,96.0,8.40,0.0,17.0,0.720,4000.0,...,False,False,False,False,False,False,False,False,False,False
1,27050.0,0.1099,55000.0,25.0,53.9,22.87,0.0,8.0,0.612,35700.0,...,False,False,False,False,False,False,False,False,False,False
2,12000.0,0.1099,60000.0,0.0,15.9,4.62,1.0,3.0,0.240,18100.0,...,False,False,False,False,False,False,False,False,False,False
3,28000.0,0.0762,325000.0,16.7,67.1,18.55,1.0,3.0,0.546,42200.0,...,False,False,False,False,False,False,False,False,False,False
4,12000.0,0.1353,40000.0,33.3,79.6,16.94,0.0,17.0,0.688,7000.0,...,False,False,False,False,False,False,False,False,False,False


In [41]:
combined_df.to_hdf('C:/Users/xiaow/work/jp_awm/interview/Take Home Project/combined_train_test_cleaned.hdf5', key='data')
aa = pd.read_hdf('C:/Users/xiaow/work/jp_awm/interview/Take Home Project/combined_train_test_cleaned.hdf5')
aa.equals(combined_df)

True